In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,recall_score,f1_score
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('mydata.csv')

In [3]:
y=df.Accident_Severity
df = df.drop(['Accident_Severity'], axis=1)

In [4]:
logistic_regression=LogisticRegression(class_weight="balanced",solver="newton-cg")

In [5]:
from sklearn.metrics import recall_score
from sklearn.metrics import  make_scorer
recall_scorer = make_scorer(recall_score,average="micro")

In [6]:
kf = KFold(n_splits=10)

In [7]:
def normalize(z):
    x = z.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(np.vstack(x))
    return x_scaled

In [8]:
for i in range(0,9):
    df.iloc[:,i]=normalize(df.iloc[:,i])

In [9]:
X = df.values

In [85]:
X_new = SelectKBest(chi2, k=4).fit_transform(X, y)

In [86]:
grid={"C":[0.00001,0.01]}

logistic_regression_cv=GridSearchCV(logistic_regression,grid,cv=10,scoring=recall_scorer)
logistic_regression_cv.fit(X_new,y)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=0.009, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='newton-cg', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1e-05, 0.01]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn',
       scoring=make_scorer(recall_score, average=micro), verbose=0)

In [87]:
print("tuned hpyerparameters :(best parameters) ",logistic_regression_cv.best_params_)

tuned hpyerparameters :(best parameters)  {'C': 1e-05}


In [81]:
# Grid search cross validation 

print("Train \t\t\t Test")
for train_index, test_index in kf.split(X_new):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y[train_index], y[test_index]
    logistic_regression=LogisticRegression(class_weight="balanced",C=0.009,solver="newton-cg")
    logistic_regression.fit(X_train,y_train)
    y_ = logistic_regression.predict(X_test)

    y_pred = logistic_regression.predict(X_train)
    
    print(accuracy_score(y_train,y_pred),"\t",accuracy_score(y_test,y_))


Train 			 Test
0.538536348164 	 0.554585152838
0.535002430724 	 0.588621444201
0.544725328148 	 0.588621444201
0.548128342246 	 0.560175054705
0.547399124939 	 0.512035010941
0.552746718522 	 0.509846827133
0.551531356344 	 0.525164113786
0.552989790958 	 0.509846827133
0.535488575596 	 0.54704595186
0.544968400583 	 0.525164113786


In [88]:
confusion_matrix(y_test,y_,labels=[1,2,3])

array([[  4,   1,   2],
       [ 19,   7,  45],
       [105,  45, 229]])